In [14]:
import sys
import torch
from torch import nn
import pandas as pd

df = pd.read_csv("../starter/loan_data.csv")
df = df[["loan_status", "person_income", "loan_intent", "loan_percent_income", "credit_score"]]
df = pd.get_dummies(df, columns=["loan_intent"])

# Create target tensor y from "loan_status"
y = torch.tensor(df["loan_status"], dtype=torch.float32)\
    .reshape((-1, 1))

# Prepare features
X_data = df.drop("loan_status", axis=1).astype('float32').values
X = torch.tensor(X_data, dtype=torch.float32)

# Normalize X
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std

# Build model: Linear(9,32) -> ReLU -> Linear(32,16) -> ReLU -> Linear(16,1)
model = nn.Sequential(
    nn.Linear(9, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
)

# Create loss function and optimizer
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_entries = X.size(0)
batch_size = 32

for i in range(0, 100):
    loss_sum = 0.0
    for start in range(0, num_entries, batch_size):
        end = min(num_entries, start + batch_size)
        X_batch = X[start:end]
        y_batch = y[start:end]

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = loss_fn(outputs, y_batch)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()

    if i % 2 == 0:
        print(loss_sum)

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X)
    y_pred = (nn.functional.sigmoid(outputs) > 0.5)
    acc = (y_pred == y).type(torch.float32).mean()
    print(acc)

778.6349059045315
624.1157028749585
615.9355404749513
613.2013979554176
611.4129536449909
609.9850221425295
608.650811560452
607.34344650805
606.0811311416328
604.8175686560571
603.6037000454962
602.4183838143945
601.2822065465152
600.1930220536888
599.1125586666167
598.1027305722237
597.0721660517156
596.0223572254181
594.9691705070436
593.9803100898862
593.0080194585025
592.1636118553579
591.3280024118721
590.5582527518272
589.8068945109844
589.1323641687632
588.4458007588983
587.8408892378211
587.2231567166746
586.6639154031873
586.0921218208969
585.5492163524032
585.0139245688915
584.4798399321735
584.0290275849402
583.5899274721742
583.1498779207468
582.6892772763968
582.2973609268665
581.924199461937
581.5666346810758
581.2373939417303
580.9148982018232
580.6095447763801
580.3383036181331
580.0488488301635
579.8088214732707
579.5418261177838
579.2995021268725
579.0642630010843
tensor(0.8219)
